## Topic Modeling with Gensim

### Gensim word2vec

 - 단어마다 차례대로 인덱싱을 하여 벡터화 하지 않고, 유사한 단어들을 비슷한 방향과 힘의 벡터를 갖도록 단어를 벡터화 시켜주는 방법 중 하나
 - `word2vec`으로 단어들을 임베딩시키면 단어끼리 연산이 가능함. 
 - 단어 간 연산이 가능하려면 단어의 벡터는 의미를 가지고 있어야 함

### pyldavis 개요
 - pyLDAvis 는 LDAvis 의 Python wrapper 로<br>
   1) topic vector 를 2 차원으로 축소하여 토픽 간의 관계를 살펴볼 수 있도록 도와줌(비슷한 좌표를 지닌 토픽들은 비슷한 맥락을 가짐)<br>
   2) 각 토픽의 키워드 제공(각 토픽에서 단어가 발생할 확률, 이를 단어의 기본 발생 확률로 정규화한 두 값 제공)

**필요한 패키지 설치**<br>
pip install gensim<br>
pip install pyldavis

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [2]:
# 필요한 library들을 import
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [3]:
def tokenize(text):
    min_length = 3 #최소 단어 크기
    # nltk의 tokenizer를 이용해서 word 추출한 후에 소문자로 변환
    words = map(lambda word: word.lower(), word_tokenize(text))
    #words = word_tokenize(text.lower()) #이렇게 해도 되는지 확인
    # stopwords 제외
    words = [word for word in words if word not in cachedStopWords]
    # portr stemmer 적용
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words)))
    #tokens = [PorterStemmer().stem(token) for token in words]  #이렇게 해도 되는지 확인
    #알파벳으로 이루어진 단어들만 추출
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    #filtered_tokens = [token for token in tokens if p.match(token) and len(token) >= min_length]   #이렇게 해도 되는지 확인
    return filtered_tokens

In [4]:
docs = [tokenize(doc) for doc in newsgroups_train.data]

In [5]:
print(newsgroups_train.data[0])
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

Gensim으로 topic modeling을 하기 위해서는 Dictionary, Corpus, Model의 3단계를 거쳐야 함

In [6]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19660
Number of unique words after removing rare and common words: 5628


In [7]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5628
Number of documents: 2034


In [8]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 25
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 40
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 1min 52s


In [9]:
#! pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [11]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
23     0.197591 -0.108881       1        1  19.863942
24     0.026158  0.156025       2        1  14.325791
14     0.222694 -0.068291       3        1  12.927218
15     0.155570  0.052870       4        1  11.148382
2      0.208931 -0.028331       5        1   7.444808
9      0.236587 -0.037349       6        1   5.789940
19     0.006546  0.157096       7        1   4.203937
7      0.003308  0.206249       8        1   3.498713
5     -0.017054  0.056851       9        1   2.406136
8      0.075571 -0.021033      10        1   2.380014
6      0.076209  0.254512      11        1   1.963019
17    -0.004054 -0.099510      12        1   1.763611
16     0.076652 -0.078869      13        1   1.683695
22    -0.132595 -0.018076      14        1   1.537419
10     0.102435 -0.211227      15        1   1.415642
20    -0.051076  0.056459      16        1   1.254795
11    -0.049583 -0.063184      17        1   0.997357
4     -0.057559  0.025728      18        1   0.941494
13    -0.068883  0.059421      19        1   0.938296
1     -0.152438 -0.042091      20        1   0.925932
12    -0.149207  0.002533      21        1   0.721081
18    -0.169628 -0.054914      22        1   0.573914
21    -0.142943 -0.055823      23        1   0.517869
0     -0.189221 -0.074257      24        1   0.478877
3     -0.204012 -0.065909      25        1   0.298120, topic_info=        Term         Freq        Total Category  logprob  loglift
214    space   896.000000   896.000000  Default  30.0000  30.0000
60       n't  1703.000000  1703.000000  Default  29.0000  29.0000
284     imag   844.000000   844.000000  Default  28.0000  28.0000
724      god   788.000000   788.000000  Default  27.0000  27.0000
309     good   568.000000   568.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
1323    slow     5.696213    16.141121  Topic25  -4.5360   4.7739
1613    plot    11.002576    92.710872  Topic25  -3.8777   3.6841
1319  liquid     3.955146    22.111141  Topic25  -4.9008   4.0944
652   commun     5.351749   130.299530  Topic25  -4.5984   2.6230
1124    road     4.234797    46.781225  Topic25  -4.8325   3.4133

[1428 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4244     12  0.958903       a.a
1484     24  0.684437      a.d.
3446     22  0.687657  a.lizard
2322     23  0.601823      a.m.
399       3  0.988763   absolut
...     ...       ...       ...
790       5  0.025382      york
3119      1  0.982307     young
3606      7  0.986347      zero
1113     17  0.736613       zip
4342     20  0.895222      zone

[2796 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[24, 25, 15, 16, 3, 10, 20, 8, 6, 9, 7, 18, 17, 23, 11, 21, 12, 5, 14, 2, 13, 19, 22, 1, 4])